# Mean Encoding Tutorial
1. In this tutorial we are going to Mean encode "Embarked" column. Using four different encoding schemes.
    * K-Fold Validation
    * Leave one out 
    * Smoothing
    * Expanding mean
2. Then we are going to find corelation in between every encoding scheme and target.

# Introduction
Mean Encoding 

# Implementation


In [1]:
# Importing libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

# Reading Dataset
Train_Data = pd.read_csv("../input/pak-ec-dataset/Pakistan Largest Ecommerce Dataset.csv")

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (1,2,3,7,8,9,11,12,13,14,17,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
# About Dataset

Train_Data.shape
#Train_Data.info()
Train_Data.nunique()

Train_Data["status"].unique()
Train_Data["qty_ordered"].isnull().sum()

Train_Data.head()
#Train_Data.tail(15

,item_id,status,created_at,sku,price,qty_ordered,grand_total,increment_id,category_name_1,sales_commission_code,...,Month,Customer Since,M-Y,FY,Customer ID,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,211131.0,complete,7/1/2016,kreations_YI 06-L,1950.0,1.0,1950.0,100147443,Women's Fashion,\N,...,7.0,2016-7,7-2016,FY17,1.0,NaN,NaN,NaN,NaN,NaN
1,211133.0,canceled,7/1/2016,kcc_Buy 2 Frey Air Freshener & Get 1 Kasual Bo...,240.0,1.0,240.0,100147444,Beauty & Grooming,\N,...,7.0,2016-7,7-2016,FY17,2.0,NaN,NaN,NaN,NaN,NaN
2,211134.0,canceled,7/1/2016,Ego_UP0017-999-MR0,2450.0,1.0,2450.0,100147445,Women's Fashion,\N,...,7.0,2016-7,7-2016,FY17,3.0,NaN,NaN,NaN,NaN,NaN
3,211135.0,complete,7/1/2016,kcc_krone deal,360.0,1.0,60.0,100147446,Beauty & Grooming,R-FSD-52352,...,7.0,2016-7,7-2016,FY17,4.0,NaN,NaN,NaN,NaN,NaN
4,211136.0,order_refunded,7/1/2016,BK7010400AG,555.0,2.0,1110.0,100147447,Soghaat,\N,...,7.0,2016-7,7-2016,FY17,5.0,NaN,NaN,NaN,NaN,NaN


In [3]:
Train_Data.dtypes

item_id                  float64
status                    object
created_at                object
sku                       object
price                    float64
qty_ordered              float64
grand_total              float64
increment_id              object
category_name_1           object
sales_commission_code     object
discount_amount          float64
payment_method            object
Working Date              object
BI Status                 object
 MV                       object
Year                     float64
Month                    float64
Customer Since            object
M-Y                       object
FY                        object
Customer ID              float64
Unnamed: 21              float64
Unnamed: 22              float64
Unnamed: 23              float64
Unnamed: 24              float64
Unnamed: 25              float64
dtype: object

In [4]:
#Drop multiple columns (Just to make dataset bit easier for demonstration)
Train_Data.drop(Train_Data.iloc[:, 14:26], axis=1, inplace=True)

In [5]:
# Dropping all rows with NaN values
Train_Data.dropna(how='all', inplace=True)

In [6]:
# Renaming the labels
Train_Data.status.replace({"complete": 1, "received": 1, "canceled": 0, "order_refunded": 0, "refund": 0, "closed": 0, "fraud":0, "holded":0, "exchange":0, "pending_paypal":0, "paid":0, "\\N":0, "cod":0, "pending":0, "processing":0,"nan":0, "payment_review":0}, inplace=True)

In [7]:
Train_Data.sort_values(['qty_ordered','status'],inplace=True)

In [8]:
Train_Data.head()
#Train_Data.shape

,item_id,status,created_at,sku,price,qty_ordered,grand_total,increment_id,category_name_1,sales_commission_code,discount_amount,payment_method,Working Date,BI Status
1,211133.0,0.0,7/1/2016,kcc_Buy 2 Frey Air Freshener & Get 1 Kasual Bo...,240.0,1.0,240.0,100147444,Beauty & Grooming,\N,0.0,cod,7/1/2016,Gross
2,211134.0,0.0,7/1/2016,Ego_UP0017-999-MR0,2450.0,1.0,2450.0,100147445,Women's Fashion,\N,0.0,cod,7/1/2016,Gross
5,211137.0,0.0,7/1/2016,UK_Namkino All In One 200 Gms,80.0,1.0,80.0,100147448,Soghaat,\N,0.0,cod,7/1/2016,Gross
8,211140.0,0.0,7/1/2016,Apple iPhone 6S 64GB,96499.0,1.0,96499.0,100147451,Mobiles & Tablets,\N,0.0,ublcreditcard,7/1/2016,Gross
9,211141.0,0.0,7/1/2016,Apple iPhone 6S 64GB,96499.0,1.0,96499.0,100147452,Mobiles & Tablets,\N,0.0,mygateway,7/1/2016,Gross


In [9]:
Train_Data.head()

,item_id,status,created_at,sku,price,qty_ordered,grand_total,increment_id,category_name_1,sales_commission_code,discount_amount,payment_method,Working Date,BI Status
1,211133.0,0.0,7/1/2016,kcc_Buy 2 Frey Air Freshener & Get 1 Kasual Bo...,240.0,1.0,240.0,100147444,Beauty & Grooming,\N,0.0,cod,7/1/2016,Gross
2,211134.0,0.0,7/1/2016,Ego_UP0017-999-MR0,2450.0,1.0,2450.0,100147445,Women's Fashion,\N,0.0,cod,7/1/2016,Gross
5,211137.0,0.0,7/1/2016,UK_Namkino All In One 200 Gms,80.0,1.0,80.0,100147448,Soghaat,\N,0.0,cod,7/1/2016,Gross
8,211140.0,0.0,7/1/2016,Apple iPhone 6S 64GB,96499.0,1.0,96499.0,100147451,Mobiles & Tablets,\N,0.0,ublcreditcard,7/1/2016,Gross
9,211141.0,0.0,7/1/2016,Apple iPhone 6S 64GB,96499.0,1.0,96499.0,100147452,Mobiles & Tablets,\N,0.0,mygateway,7/1/2016,Gross


In [10]:
Train_Data.dtypes

item_id                  float64
status                   float64
created_at                object
sku                       object
price                    float64
qty_ordered              float64
grand_total              float64
increment_id              object
category_name_1           object
sales_commission_code     object
discount_amount          float64
payment_method            object
Working Date              object
BI Status                 object
dtype: object

In [11]:
Train_Data["qty_ordered"].nunique()
Train_Data["qty_ordered"].unique()

array([   1.,    2.,    3.,    4.,    5.,    6.,    7.,    8.,    9.,
         10.,   11.,   12.,   13.,   14.,   15.,   16.,   17.,   18.,
         19.,   20.,   21.,   22.,   23.,   24.,   25.,   26.,   27.,
         28.,   29.,   30.,   31.,   32.,   33.,   34.,   35.,   36.,
         37.,   38.,   40.,   45.,   48.,   49.,   50.,   52.,   56.,
         60.,   66.,   70.,   72.,   75.,   90.,   96.,   97.,   99.,
        100.,  102.,  119.,  120.,  125.,  130.,  141.,  144.,  150.,
        180.,  185.,  186.,  187.,  200.,  300.,  304.,  380.,  500.,
        502., 1000.])

In [12]:
QO_Globalmean = Train_Data["qty_ordered"].mean()

In [13]:
QO_Globalmean

1.2963881722564001

# Mean Encoding with K-Fold Validation
***Steps***
1. **Import K-Fold with K=5 and shuffle = False**
2. **Split Train data in to Tr_d and Val_d**
3. **Iterate through K folds (If K=x This iteration will repeat x times)**
    * Train and Validation -(Assign)- train and validation index
    * Find Target mean for Train Folds groupedby(Categorical feature under operation)
    * New Feature in Validation Fold -(Assign)- Validation fold (Categorical feature under operation) .map(Target mean of Train folds)
    * Dataset(Index val) -(Assign)- Validation fold
4. **Fill null values with global mean in new feature of Dataset**

In [14]:
#from sklearn.model_selection import KFold

#kf = KFold(n_splits=5, shuffle=False)

#for index_train, index_valid in kf.split(Train_Data):
#    X_tr, X_val = Train_Data.iloc[index_train], Train_Data.iloc[index_valid]
    
    # target coding of valid dataset depends on train dataset
#    X_tr_group = X_tr.groupby('Month')['status'] 
#    X_val['item_target_enc'] = X_val['Month'].map(X_tr_group.mean())
    
    # copy target encoding back to all_data
#    Train_Data.iloc[index_valid] = X_val
    

Train_Data.head()
#Train_Data['item_target_enc'].fillna(QO_Globalmean, inplace=True) 
#encoded_feature = Train_Data['item_target_enc'].values


,item_id,status,created_at,sku,price,qty_ordered,grand_total,increment_id,category_name_1,sales_commission_code,discount_amount,payment_method,Working Date,BI Status
1,211133.0,0.0,7/1/2016,kcc_Buy 2 Frey Air Freshener & Get 1 Kasual Bo...,240.0,1.0,240.0,100147444,Beauty & Grooming,\N,0.0,cod,7/1/2016,Gross
2,211134.0,0.0,7/1/2016,Ego_UP0017-999-MR0,2450.0,1.0,2450.0,100147445,Women's Fashion,\N,0.0,cod,7/1/2016,Gross
5,211137.0,0.0,7/1/2016,UK_Namkino All In One 200 Gms,80.0,1.0,80.0,100147448,Soghaat,\N,0.0,cod,7/1/2016,Gross
8,211140.0,0.0,7/1/2016,Apple iPhone 6S 64GB,96499.0,1.0,96499.0,100147451,Mobiles & Tablets,\N,0.0,ublcreditcard,7/1/2016,Gross
9,211141.0,0.0,7/1/2016,Apple iPhone 6S 64GB,96499.0,1.0,96499.0,100147452,Mobiles & Tablets,\N,0.0,mygateway,7/1/2016,Gross


In [15]:
Train_Data.head()

,item_id,status,created_at,sku,price,qty_ordered,grand_total,increment_id,category_name_1,sales_commission_code,discount_amount,payment_method,Working Date,BI Status
1,211133.0,0.0,7/1/2016,kcc_Buy 2 Frey Air Freshener & Get 1 Kasual Bo...,240.0,1.0,240.0,100147444,Beauty & Grooming,\N,0.0,cod,7/1/2016,Gross
2,211134.0,0.0,7/1/2016,Ego_UP0017-999-MR0,2450.0,1.0,2450.0,100147445,Women's Fashion,\N,0.0,cod,7/1/2016,Gross
5,211137.0,0.0,7/1/2016,UK_Namkino All In One 200 Gms,80.0,1.0,80.0,100147448,Soghaat,\N,0.0,cod,7/1/2016,Gross
8,211140.0,0.0,7/1/2016,Apple iPhone 6S 64GB,96499.0,1.0,96499.0,100147451,Mobiles & Tablets,\N,0.0,ublcreditcard,7/1/2016,Gross
9,211141.0,0.0,7/1/2016,Apple iPhone 6S 64GB,96499.0,1.0,96499.0,100147452,Mobiles & Tablets,\N,0.0,mygateway,7/1/2016,Gross
